In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
time: 704 µs (started: 2023-05-01 11:20:55 +00:00)


In [2]:
from fastkaggle import *
from fastai.vision.all import *
from fasterai.sparse.all import *
from torchvision.models import vgg16_bn

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")


time: 3.69 s (started: 2023-05-01 11:20:55 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 17.7 s (started: 2023-05-01 11:20:59 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ), 
                                   bs=32)

time: 5.44 s (started: 2023-05-01 11:21:17 +00:00)


In [5]:
def run_once(func):
    """
    A decorator that runs a function only once and returns the result for all subsequent calls.
    """
    memo = {}
    def wrapper(*args, **kwargs):
        if func not in memo:
            memo[func] = func(*args, **kwargs)
        return memo[func]
    return wrapper


time: 1.3 ms (started: 2023-05-01 11:21:22 +00:00)


In [6]:
@run_once
def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size 
    print('Model size: {} Byte'.format(model_size))


time: 54.4 ms (started: 2023-05-01 11:21:22 +00:00)


In [7]:
class MemoryCallback(Callback):
    def before_batch(self):
        get_model_file_size(self.learn.model)

time: 83.7 ms (started: 2023-05-01 11:21:22 +00:00)


In [8]:
sp_cb=[
      # GradientAccumulation(32), 
      SparsifyCallback(sparsity=50, 
                         granularity='filter', 
                         context='local', 
                         criteria=large_final, 
                         schedule=one_cycle
                         ),
      MemoryCallback(),
]

time: 12.5 ms (started: 2023-05-01 11:21:22 +00:00)


In [9]:
model = vgg16_bn(num_classes=10)

time: 3.28 s (started: 2023-05-01 11:21:22 +00:00)


In [10]:
learn = Learner(dls, 
                model, 
                metrics=[accuracy,
                         error_rate,
                        ])
learn.unfreeze()
learn.recorder.train_metrics = True

time: 9.17 ms (started: 2023-05-01 11:21:26 +00:00)


In [11]:
learn.fit_one_cycle(1, 3e-5, cbs=sp_cb)


Pruning of filter until a sparsity of [50]%
Saving Weights at epoch 0


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.176666,0.202724,0.797276,2.076549,0.261538,0.738462,02:11


Model size: 537239848 Byte
Sparsity at the end of epoch 0: [50.0]%
Final Sparsity: [50.0]%
Sparsity in Conv2d 2: 50.00%
Sparsity in Conv2d 5: 50.00%
Sparsity in Conv2d 9: 50.00%
Sparsity in Conv2d 12: 50.00%
Sparsity in Conv2d 16: 50.00%
Sparsity in Conv2d 19: 50.00%
Sparsity in Conv2d 22: 50.00%
Sparsity in Conv2d 26: 50.00%
Sparsity in Conv2d 29: 50.00%
Sparsity in Conv2d 32: 50.00%
Sparsity in Conv2d 36: 50.00%
Sparsity in Conv2d 39: 50.00%
Sparsity in Conv2d 42: 50.00%
time: 2min 11s (started: 2023-05-01 11:21:26 +00:00)
